In [5]:
using DrWatson
@quickactivate

# Hunting for Maximum Current as Easily as Possible

For fixed voltage, we have that expected current should be proportional to the expectation of being in the open state at time $t$:

$\qquad \displaystyle
    \left\langle I(t) \right \rangle \propto \left \langle \mu(t) \mid \delta_\mathrm{O} \right \rangle
$

$\qquad \displaystyle \phantom{\left\langle I(t) \right \rangle} =
    \left\langle{\mu_0} \right| k \mathrm{e}^{(t - \tau)\mathbf{G}_{v}} \left| \delta_\mathrm{O} \right\rangle
$

$\qquad \displaystyle \phantom{\left\langle I(t) \right \rangle} =
    k\sum_i [\mu_0]_i \, [\mathrm{e}^{(t - \tau)\mathbf{G}_{v}}]_{i\delta_\mathrm{O}}
$,

where $\delta_\mathrm{O}$ is the index corresponding to the "open" state, $\tau$ is the time at which we switched to a fixed voltage $v$, and $k$ captures all the evolution up to time $\tau$. We wish to *maximize* this function by varying $t > \tau$. 

So, let's differentiate with respect to $t$ and set the result to $0$:

$\qquad \displaystyle
\frac{\mathrm{d}}{\mathrm{d}t} \left\langle I(t) \right \rangle \propto 
    k\sum_i [\mu_0]_i \, \frac{\mathrm{d}}{\mathrm{d}t} [\mathrm{e}^{(t - \tau)\mathbf{G}_{v}}]_{i\delta_\mathrm{O}} = 0
$,

where we can take the derivative inside without issue since we're dealing with a finite sum. But now the issue is we're taking the derivative of an element of a matrix determined from an infinite sum.

## Directions
* Numerics: create a Julia function that takes in
    1. A "time" (difference) $x$,
    1. The rate matrix $\mathbf{G}$,
    1. The index $i$ of an arbitrary row/column 
    1. The index $\delta_\mathrm{O}$ of the row/column corresponding to the open state
    
    and returns $[\mathrm{e}^{x\mathbf{G}}]_{i\delta_\mathrm{O}}$. 
    
    If $\left \langle \mu_0 \right|$ has strictly positive elements (which we can set up, and which will certainly be true of any "real" $\left \langle \mu(t) \right|$ for irreducible $\mathbf{G}$, which we expect most of the time and require for a unique stationary distribution) then $\left\langle I(t) \right \rangle = 0$ only if $\frac{\mathrm{d}}{\mathrm{d}t}[\mathrm{e}^{x\mathbf{G}}]_{i\delta_\mathrm{O}} = 0$ **for each $i$ in the state index**.

    So, via the Julia function specified above, we can try to just numerically probe maxima/minima of one $i$ at a time, knowing that the local minimum we're after must correspond to a local max, min, or flat region for each $i$ simultaneously.